# Weird descriptions

Ever come across a newspaper headline that makes you think someone was just sticking random words together, but it turns out it was a real (if somewhat unusual) event? This notebook takes inspiration from those headlines, and runs through the steps to find weird descriptions in Discovery series.

As with always, start by importing the necessary libraries. 

In [ ]:
%pip install -q json
%pip install -q wordfreq
%pip install -q matplotlib
%pip install -q numpy
import json
from wordfreq import word_frequency
import matplotlib.pyplot as plt
import numpy as np

import helper_functions as hf

## What is "weird" anyway?

The first step is to define metrics to measure weirdness. The inspiration of weird headlines, in particular with words that are used infrequently and in unusual combinations. The main series of notebooks showed, its fairly straightforwards to get all the descriptions from a series, so lets assume access to that data. This gives us a two main areas we can use: scoring based on how often words or combinations of words appear in the dataset, and scoring based on how many records a word appears in. 

#### Weirdness based on word frequency

`record weirdness = average(average word weirdness from local, average word weirdness from global, average weirdness of word combinations)`

Where: 
 
`word weirdness from local = 1 - (word frequency in all records in the series / total number of words in all records in the series)` - As a hypothetical example, if all records start "The", then have 100 different words that don't appear anywhere else, "the" should have a low score - its uncommon within individual records, but very common overall.

`word weirdness from global` - There are datasets that provide word frequency in the English language which can be used to get a sense of how common a word is in general. 

`weirdness of word combinations = 1 - (pair frequency in all records in the series / toal number of pairs in the series)` - Looking at every adjacent pair of words in every record, how often are those two words next to each other? Similar to the word weirdness, if all descriptions start with "The Parliament", then have 100 different words that don't appear in other records, "The Parliament" should have a low score.


These values are then averaged to accomodate for the length variability of record descriptions - some have lots of words, some have very few. If the weirdness value for each word was summed, then longer records would have a higher weirdness score than shorter records.

#### Document occurrence weirdnes

`record weirdness = average(document frequency weirdness of all words in the record)` - Documents with a higher average weirdness score with this metric are ones that contain words that appear in fewer records.

Where:

`document frequency weirdness of a word = 1 - (total number of records the word appears in/total number of documents)` - This is a measure of how many records a word appears in, with a higher score for words that appear in fewer records.

Similar to the weirdness from global, the document frequency weirdness could be calculated from a larger dataset - for example, all records in the National Archives - before being used to calculate a record's weirdness.

The key difference between this approach and weirdness based on word frequency is that records with a single word repeated frequently won't skew the score as much. For example, imagine a series of 100 records. If 50% of them used the word "the" once, and one of them had a really long description with the word "parliament" in it 100 times, the average weirdness from word frequency for both words would be the same. The document occurence weirdness, however, would be much higher for "parliament" than "the" - as it only accounts for the number of records a word appears in. Depending on the dataset, this may be a more reliable measure of weirdness. It will certainly be interesting to compare the two later on.

## Lets start with getting the data

As it was explained in detail in the [main series,](../1-intro-to-discovery-api.ipynb) this step of the process is in the `helper_functions.py` file to avoid repitition and making this notebook full of familiar code. Simply run the cell below to get the data - the default series it'll use is [`TITH`](https://discovery.nationalarchives.gov.uk/details/r/C254) but you can change that if you want. Note that really large series (eg `ADM`) will take a while to run, and may result in a timeout error from too many requests to the API. Note also that this cell is also going to print the current c-code its working on - this is to give a visual indication that it is running. 

To make it easy to work through and work on the weirdness score, the data gathered from this first step is going to look like this (note the nested nature as we are gathering every record from a series - sub-series will be stepped into using the child endpoint of the API):

```JSON
{
    "series": "TITH",
    "total_records_count": 100,
    "children": [
        {
            "series": "TITH 1",
            "id": "discovery web id",
            "description": "description of the record",
            "children" : [
                {
                    "series": "TITH 1/1",
                    "id": "discovery web id",
                    "description": "description of the record"
                    "children": [ # and so on]
                }
            ]
        }
    ]
}
```

In [ ]:
data = hf.get_series_description("TITH")

print(json.dumps(data, indent=4))

## Scoring the weirdness

### Weirdness of words (scoring based on word frequency)

With the data gathered, lets start working on the score. The first step is to see what words are in the descriptions, their frequnecy, and thus their weirdness score. The new data will be stored a new key to the data structure `word_weirdness` at the same level as the `children` key. The data in it will look like this:

```JSON
{
    "series": "TITH",
    "children": [as before],
    "total_words": 1000,
    "word_weirdness": [
        {
            "word": "the",
            "occurences": 100,
            "weirdness": 0.9,
            "word_frequency": 0.1
        },
        {for every word in the description}
    ]
}
```

Thanks to some builtins in Python, this is a fairly simple process. The first, and most complex step, is to build a single long list of every word in every description in the series. The complexity here is from the nested descriptions, hence the need for a recursive function. Once we have this list, the use of `set()` means we are looping throught the unique words in the list, and `.count()` counts the number of times each word appears in the main list.

Word frequency, in this case, is being supplied by the [`wordfreq`](https://github.com/rspeer/wordfreq) package, which has a list of word frequencies in the English language, based on similar maths using a much larger dataset. You can read more about it in the link.

These scores are based on all the descriptions in the data, not the descriptions of individual records, hence looping through every record and keeping a total score, rather than resetting the score each time. After this, the process for word pairs will be similar. 


In [ ]:
words = []
word_weirdness = []

def get_words(description):
    if description["children"] != "No children":
        for child in description["children"]:
            get_words(child) # Recursion - if the record has children, call this function again with the child as the argument
    if "description" in description: # The top level of the data doesn't have a description
        if description["description"] != None: # Sometimes the description is empty
            for word in description["description"].split(" "):
                words.append(word)
    return

get_words(data)

for unique_word in set(words):
    word_weirdness.append(
        {
            "word": unique_word,
            "count": words.count(unique_word),
            "weirdness" : 1 - (words.count(unique_word) / len(words)),
            "word_frequency": word_frequency(unique_word, "en", wordlist="large")
        }
    )

word_weirdness.sort(key=lambda x: x["weirdness"], reverse=True) # This sorts the list in place, with the highest weirdness first

data["total_words"] = {
    "total": len(words),
    "unique": len(set(words))
}
data["word_weirdness"] = word_weirdness

print(json.dumps(data["word_weirdness"], indent=4))

If you scroll through the results of that cell, you'll notice that the data have an very [long tail](https://en.wikipedia.org/wiki/Long_tail), plotting that later will allow a better look at the data. Its also interesting to note that, as word frequency scores are based on the English language, many of these words results in really tiny scores.

### Document occurance weirdness

Now we have a list of the unique words, we can calculate the document occurance weirdness, and add that to the data structure. This will be stored as the key `document_occurance_weirdness` along with the weirdness for each word. In this case, for each unique word we found we want to loop through each record description, see what words it has, and add 1 to the score if the unique word appears. Note that we don't add to the score if a word appears twice in a description - we're looking for how many descriptions the word appears in, so we only want to count it once. 

In [ ]:
weirdness_score = 0

def idf_calculator(record, word):
    global weirdness_score
    if record["children"] != "No children":
        for child in record["children"]:
            idf_calculator(child, word)
    if "description" in record:
        if record["description"] != None:
            if word in record["description"].split(" "):
                weirdness_score += 1
    return weirdness_score

for word in data["word_weirdness"]:
    weirdness_score = idf_calculator(data, word["word"])
    total_records_count = data["total_records_count"]
    data["word_weirdness"][data["word_weirdness"].index(word)]["document_occurance_weirdness"] = 1-(weirdness_score/total_records_count)
    weirdness_score = 0

print(json.dumps(data["word_weirdness"], indent=4))

## Word pairs

This is a similar process to the above, with the main switch being that instead of isolating every word, we are isolating every pair. The output data structure is going to look essentially the same. 


In [ ]:
pairs = []
pair_weirdness = []

def get_pairs(description):
    if description["children"] != "No children":
        for child in description["children"]:
            get_pairs(child) # Recursion - if the record has children, call this function again with the child as the argument
    if "description" in description: # The top level of the data doesn't have a description
        if description["description"] != None: # Sometimes the description is empty
            for i in range(len(description["description"].split(" ")) - 1):
                pairs.append((description["description"].split(" ")[i], description["description"].split(" ")[i+1]))
    return

get_pairs(data)

for unique_pair in set(pairs):
    pair_weirdness.append(
        {
            "pair": unique_pair,
            "count": pairs.count(unique_pair),
            "weirdness" : 1 - (pairs.count(unique_pair) / len(pairs))
        }
    )

pair_weirdness.sort(key=lambda x: x["weirdness"], reverse=True) # This sorts the list in place, with the highest weirdness first

data["pair_weirdness"] = pair_weirdness

print(json.dumps(data["pair_weirdness"], indent=4))

The big thing to notice here is that almost every pair is fairly unique, getting a score close to 1. This is to be expected, with the number of possible pairs being so large. For example, for 100 unique words, there would be 100! (100 factorial) possible pairs. This is a very large number, and a description will only have a tiny fraction of them.

## Adding data to the original structure

Now scores for each word have been calculated, the weirdness score for each record (using the definitions earlier in the notebook) can be added to the data structure. They will be added in as new key/values for each record. Given that the different scores for weirdness are so different, the average of the three, and the three scores themselves are included. Remember that the scores are being averaged to accomodate for varying description lengths. 

Calculating a records document frequency weirdness is slightly more complex, needing its own function. This is simly as we need to loop through every word in the record as well as every word in the description to get the score.

In [ ]:
def record_idf_calculator(description):
    current_weirdness_score = 0
    for word in description.split(" "):
        for word_weirdness in data["word_weirdness"]:
            if word_weirdness["word"] == word:
                current_weirdness_score += word_weirdness["document_occurance_weirdness"]
    averaged_weirdness_score = current_weirdness_score / len(description.split(" "))
    return averaged_weirdness_score

def add_description_scores(record):
    if record["children"] != "No children":
        for child in record["children"]:
            add_description_scores(child)
    if "description" in record:
        if record["description"] != None:
            record["word_weirdness_score"] = (sum([word["weirdness"] for word in word_weirdness if word["word"] in record["description"].split(" ")])/len(record["description"].split(" ")))
            record["word_frequency_weirdness_score"] = (sum([word["word_frequency"] for word in word_weirdness if word["word"] in record["description"].split(" ")])/len(record["description"].split(" ")))
            record["pair_weirdness_score"] = (sum([pair["weirdness"] for pair in pair_weirdness if pair["pair"] in [(record["description"].split(" ")[i], record["description"].split(" ")[i+1]) for i in range(len(record["description"].split(" ")) - 1)]])/len(record["description"].split(" ")))/2
            record["average_weirdness_score"] = (record["word_weirdness_score"] + record["word_frequency_weirdness_score"] + record["pair_weirdness_score"])/3
            record["document_frequncy_score"] = record_idf_calculator(record["description"])

add_description_scores(data)

print(json.dumps(data, indent=4))

## Exploring the data

To start exploring the data, the first thing is to show the most and least weird records are. This is done by a similar recursive function, looking for the highest and lowest scores for average weirdness from both word frequency and inverse document frequency.

In [ ]:
current_weirdness = 0
weirdest_record_by_document_frequency = {}
weirdest_record_by_average_weirdness = {}
least_weird_record_by_document_frequency = {}
least_weird_record_by_average_weirdness = {}

def find_weirdest_record(record, weridness_type, direction):
    global current_weirdness
    global weirdest_record_by_document_frequency
    global weirdest_record_by_average_weirdness
    global least_weird_record_by_document_frequency
    global least_weird_record_by_average_weirdness
    if direction == "highest":
        current_weirdness = 0
    elif direction == "lowest":
        current_weirdness = float('inf')
    
    if record["children"] != "No children":
        for child in record["children"]:
            find_weirdest_record(child, weridness_type, direction)

    if "description" in record and record["description"] != None:
        if weridness_type == "document_frequncy_score":
            if direction == "highest" and record["document_frequncy_score"] > current_weirdness:
                current_weirdness = record["document_frequncy_score"]
                weirdest_record_by_document_frequency = record
            elif direction == "lowest" and record["document_frequncy_score"] < current_weirdness:
                current_weirdness = record["document_frequncy_score"]
                least_weird_record_by_document_frequency = record
        elif weridness_type == "average_weirdness_score":
            if direction == "highest" and record["average_weirdness_score"] > current_weirdness:
                current_weirdness = record["average_weirdness_score"]
                weirdest_record_by_average_weirdness = record
            elif direction == "lowest" and record["average_weirdness_score"] < current_weirdness:
                current_weirdness = record["average_weirdness_score"]
                least_weird_record_by_average_weirdness = record

find_weirdest_record(data, "document_frequncy_score", "highest")
print("The weirdest record by document frequency score is:")
print(json.dumps(hf.filter_children(weirdest_record_by_document_frequency), indent=4))

find_weirdest_record(data, "average_weirdness_score", "highest")
print("\nThe weirdest record by average weirdness score is:")
print(json.dumps(hf.filter_children(weirdest_record_by_average_weirdness), indent=4))

find_weirdest_record(data, "document_frequncy_score", "lowest")
print("The least weird record by document frequency score is:")
print(json.dumps(hf.filter_children(least_weird_record_by_document_frequency), indent=4))

find_weirdest_record(data, "average_weirdness_score", "lowest")
print("\nThe least weird record by average weirdness score is:")
print(json.dumps(hf.filter_children(least_weird_record_by_average_weirdness), indent=4))



Remember the long tail in the word count scores? Lets plot that to see how long it is. 

In [ ]:
word_and_count = {}

for word in data["word_weirdness"]:
    word_and_count[word["word"]] = word["count"]

plt_word_counts, ax_word_counts = plt.subplots(figsize=(30, 30))
ax_word_counts.bar(word_and_count.keys(), word_and_count.values())
ax_word_counts.set_xlabel('Word')
ax_word_counts.set_ylabel('Count')
ax_word_counts.set_title('Word count')
ax_word_counts.set_xticks([]) # This stops the x-ticks from being displayed - there are too many words to display them all
ax_word_counts.yaxis.set_minor_locator(plt.MultipleLocator(1))
ax_word_counts.yaxis.set_minor_formatter(plt.FuncFormatter(lambda x, _: int(x)))
ax_word_counts.grid(which="minor", axis="y", linestyle="--")
plt.show()

Yep, that's a long tail! A lot of words appear only once, with most appearing fewer than 10 times. Given that we have this data, what are the most common words?

In [ ]:
print(json.dumps(data["word_weirdness"][-10:], indent=4)) # we sorted with the highest weirdness first, so the last 10 are the least weird - and therefore the most common

To see how the weirdness score is distributed, lets plot that too. Here we're going to use a histogram, but with less refinement than other graphs, the goal is an overview of distribution of the scores. Remember that the record data is nested, so we'll need to step into the sub-series to get the scores.

In [ ]:
id_and_weirdness = {}

def get_id_and_weirdness(record):
    if record["children"] != "No children":
        for child in record["children"]:
            get_id_and_weirdness(child)
    if "id" in record:
        if "average_weirdness_score" in record:
            id_and_weirdness[record["id"]] = record["average_weirdness_score"]

get_id_and_weirdness(data)

plt_id_and_weirdness, ax_id_and_weirdness = plt.subplots(figsize=(10, 10))

ax_id_and_weirdness.bar(id_and_weirdness.keys(), id_and_weirdness.values())
ax_id_and_weirdness.set_xlabel('ID')
ax_id_and_weirdness.set_xticks([])
ax_id_and_weirdness.set_ylabel('Weirdness')
ax_id_and_weirdness.set_title('Weirdness by ID')
plt.show()

For the default series, this shows that most records are roughly the same weirdness. 

Lets see if that holds true for the document frequency weirdness, by plotting that too.

In [ ]:
id_and_document_frequncy = {}

def get_id_and_document_frequncy(record):
    if record["children"] != "No children":
        for child in record["children"]:
            get_id_and_document_frequncy(child)
    if "id" in record:
        if "document_frequncy_score" in record:
            id_and_document_frequncy[record["id"]] = record["document_frequncy_score"]

get_id_and_document_frequncy(data)

plt_id_and_document_frequncy, ax_id_and_document_frequncy = plt.subplots(figsize=(10, 10))

ax_id_and_document_frequncy.bar(id_and_document_frequncy.keys(), id_and_document_frequncy.values())
ax_id_and_document_frequncy.set_xlabel('ID')
ax_id_and_document_frequncy.set_xticks([])
ax_id_and_document_frequncy.set_ylabel('Document Frequency')
ax_id_and_document_frequncy.set_title('Document Frequency by ID')
plt.show()

# Some starter stats

The weirdest record has been found, but a quick statistical analysis will give more confidence in saying "its weird". Lets find out the mean and standard deviation of the scores. If our record is more than 3 standard deviations from the mean, then it can be said to be statistically weird. As these are very widely used metrics, `numpy` makes this very approachable.

In [ ]:
mean_weirdness = np.mean(list(id_and_weirdness.values()))
standard_deviation_weirdness = np.std(list(id_and_weirdness.values()))

print(f"The mean weirdness is {mean_weirdness} and the standard deviation is {standard_deviation_weirdness}. A record would need an average weirdness score of above {mean_weirdness + 3 * standard_deviation_weirdness} or below { mean_weirdness - 3 * standard_deviation_weirdness} to be an outlier.")

# Reprint the weirdest record

print(json.dumps(hf.filter_children(weirdest_record_by_average_weirdness), indent=4))

# see if the weirdest record is out by more than 3 standard deviations, else print the average weirdness score it would need to be an outlier

if weirdest_record_by_average_weirdness["average_weirdness_score"] > mean_weirdness + 3 * standard_deviation_weirdness or weirdest_record_by_average_weirdness["average_weirdness_score"] < mean_weirdness - 3 * standard_deviation_weirdness:
    print("The weirdest record is an outlier")
else:
    print(f"The weirdest record is not an outlier, it would need an average weirdness score of {mean_weirdness + 3 * standard_deviation_weirdness} or { mean_weirdness - 3 * standard_deviation_weirdness} to be one.")

The graph looked more varied with the document frequency weirdness, so lets see if the stats are different.

In [ ]:
mean_document_frequncy = np.mean(list(id_and_document_frequncy.values()))
standard_deviation_document_frequncy = np.std(list(id_and_document_frequncy.values()))

print(f"The mean document frequency is {mean_document_frequncy} and the standard deviation is {standard_deviation_document_frequncy}. A record would need a document frequency score of above {mean_document_frequncy + 3 * standard_deviation_document_frequncy} or below { mean_document_frequncy - 3 * standard_deviation_document_frequncy} to be an outlier.")

print(json.dumps(hf.filter_children(weirdest_record_by_document_frequency), indent=4))

if weirdest_record_by_document_frequency["document_frequncy_score"] > mean_document_frequncy + 3 * standard_deviation_document_frequncy or weirdest_record_by_document_frequency["document_frequncy_score"] < mean_document_frequncy - 3 * standard_deviation_document_frequncy:
    print("The weirdest record is an outlier")
else:
    print(f"The weirdest record is not an outlier, it would need a document frequency score of {mean_document_frequncy + 3 * standard_deviation_document_frequncy} or { mean_document_frequncy - 3 * standard_deviation_document_frequncy} to be one.")


# Saving the data

After a lot of analysis work, it makes sense to save the data. We're going to save it as a JSON file, as that's the format we've been working with. 

In [ ]:
# save the data to a file called data.json

with open("data.json", "w") as file:
    json.dump(data, file, indent=4)

## Possible next steps

There are a couple of things we could do to improve these analysis: 

- Improve the formatting of the input text. If you look at the full list of words, you'll see duplicates where the same word is written in different cases, with punctuation, or often with HTML tags. Cleaning this up will give a more accurate picture of the data.
- Removing common words. Words such as "the", "and", and "of", as examples, are all very common in English and will be standing out in the resulting scores. These are often called "stop-words". There are lists avaiable online, and an example where they are removed prior to analysis can be found in the [main series live demo notebook](../discovery.ipynb). 
- Using larger amounts of data. With limitations in place from the API, this notebook uses data from a single, small series. Compared to the rest of Discovery, this series could be highly unusual, or very normal. Using data from multiple series, or even the entire dataset may give a very different result.
- Normalising the scores. The scores are all between 0 and 1, but the scales are different. Normalising the scores would allow for a more direct comparison between the different scores.

These next steps would not change the process of the analysis, but would change the results. Before doing any of these, it is worth considering if they work towards the answer you are looking for. For example, making all words the same case looses sensitivity to different uses of the same word - it may be that a word is used as a proper noun in only one record; depending on your research question, this may be important.

# Conclusion

In this notebook, we've gone through the full process of:
1. From an inspiration, defining a metric to analyse our input data
2. Gathering the correct parts of our data
3. Calculating our metric for each record
4. Visualising the results
5. A little bit of statistical analysis
6. Saving the data
7. Considering next steps

Or, what appeared to be a light hearted look at descriptions was sneakily a full data analysis project (if a small one). Hopefully this notebook has given you a good idea of how to approach a similar problem, and you had fun along the way.